In [1]:
!pip install -q groq langchain langchain-core langchain-groq chromadb pypdf gradio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.6.3 which is incompatible.


In [4]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 M

In [9]:
pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 12.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 12.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [33]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community import embeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import time
import textwrap
import gradio as gr

In [34]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page in document:
        text += page.get_text()
    return text

file_path = '/Users/kiwitech/Downloads/1Bit Quantization in LLMs.pdf'
pdf_text = extract_text_from_pdf(file_path)

In [35]:
pdf_text

'The Era of 1-bit LLMs:\nAll Large Language Models are in 1.58 Bits\nShuming Ma∗ Hongyu Wang∗ Lingxiao Ma\nLei Wang\nWenhui Wang\nShaohan Huang\nLi Dong\nRuiping Wang\nJilong Xue\nFuru Wei⋄\nhttps://aka.ms/GeneralAI\nAbstract\nRecent research, such as BitNet [WMD+23], is paving the way for a new era of 1-\nbit Large Language Models (LLMs). In this work, we introduce a 1-bit LLM variant,\nnamely BitNet b1.58, in which every single parameter (or weight) of the LLM is\nternary {-1, 0, 1}. It matches the full-precision (i.e., FP16 or BF16) Transformer\nLLM with the same model size and training tokens in terms of both perplexity\nand end-task performance, while being significantly more cost-effective in terms\nof latency, memory, throughput, and energy consumption. More profoundly, the\n1.58-bit LLM defines a new scaling law and recipe for training new generations of\nLLMs that are both high-performance and cost-effective. Furthermore, it enables\na new computation paradigm and opens the do

In [36]:
def split_text_into_chunks(text, chunk_size, chunk_overlap):
    """
    Splits a text into chunks with specified size and overlap.
    
    Args:
        text (str): The text to be split.
        chunk_size (int): The desired chunk size.
        chunk_overlap (int): The number of characters to overlap between chunks.
        
    Returns:
        list of str: A list containing the text chunks.
    """
    chunks = []
    start = 0
    end = chunk_size
    while start < len(text):
        # Extract the chunk
        chunk = text[start:end]
        chunks.append(chunk)
        # Update start and end taking into account the overlap
        start += chunk_size - chunk_overlap
        end = start + chunk_size
    return chunks

# Example usage
chunk_size = 1000
chunk_overlap = 200
pdf_text = pdf_text  # This should be your extracted PDF text
chunks = split_text_into_chunks(pdf_text, chunk_size, chunk_overlap)



In [37]:
chunks

['The Era of 1-bit LLMs:\nAll Large Language Models are in 1.58 Bits\nShuming Ma∗ Hongyu Wang∗ Lingxiao Ma\nLei Wang\nWenhui Wang\nShaohan Huang\nLi Dong\nRuiping Wang\nJilong Xue\nFuru Wei⋄\nhttps://aka.ms/GeneralAI\nAbstract\nRecent research, such as BitNet [WMD+23], is paving the way for a new era of 1-\nbit Large Language Models (LLMs). In this work, we introduce a 1-bit LLM variant,\nnamely BitNet b1.58, in which every single parameter (or weight) of the LLM is\nternary {-1, 0, 1}. It matches the full-precision (i.e., FP16 or BF16) Transformer\nLLM with the same model size and training tokens in terms of both perplexity\nand end-task performance, while being significantly more cost-effective in terms\nof latency, memory, throughput, and energy consumption. More profoundly, the\n1.58-bit LLM defines a new scaling law and recipe for training new generations of\nLLMs that are both high-performance and cost-effective. Furthermore, it enables\na new computation paradigm and opens the d

In [38]:


#Assuming `from_texts` exists and works directly with a list of strings
vectorstore = Chroma.from_texts(
    texts=[chunk for chunk in chunks],  # Directly use the list of text chunks
    collection_name="ollama_embeds",
    embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'),
)
retriever = vectorstore.as_retriever()

In [39]:
groq_api_key = "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"

In [40]:
llm = ChatGroq(
            groq_api_key=groq_api_key,
            model_name='mixtral-8x7b-32768'
    )

In [41]:
rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [42]:
# Test the architecture with a simple hard coded question
response = rag_chain.invoke("What is this document about")
print(textwrap.fill(response, width=80))

The document is about the advantages and applications of low-bit language models
(LLMs), specifically 1.58-bit LLMs, in terms of memory usage, energy
consumption, and computational power. It also discusses the potential of 1-bit
Mixture-of-Experts (MoE) LLMs and the development of new hardware for LLMs. The
document also mentions several research papers and articles related to LLMs.


In [44]:
# Make the questions dynamic using a chat interface. Let's use gradio for this.
def process_question(user_question):
    start_time = time.time()

    # Directly using the user's question as input for rag_chain.invoke
    response = rag_chain.invoke(user_question)

    # Measure the response time
    end_time = time.time()
    response_time = f"Response time: {end_time - start_time:.2f} seconds."

    # Combine the response and the response time into a single string
    full_response = f"{response}\n\n{response_time}"

    return full_response

# Setup the Gradio interface
iface = gr.Interface(fn=process_question,
                     inputs=gr.Textbox(lines=2, placeholder="Type your question here..."),
                     outputs=gr.Textbox(),
                     title="Kiwitech LPU-RAG",
                     description="Ask any question about your document, and get an answer along with the response time.")

# Launch the interface
iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
